# Download IMDB Data 
This note book is to download and save imdb data 

In [75]:
import sys
import os
import urllib.request
import tarfile
import zipfile

import os
import glob

import nltk
nltk.download('stopwords')



[nltk_data] Downloading package stopwords to
[nltk_data]     /home/dsxuser/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [44]:
import requests
from pathlib import Path

In [21]:
url = 'http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'
r = requests.get(url, allow_redirects=True)

In [23]:
open('aclImdb_v1.tar.gz', 'wb').write(r.content)

84125825

In [30]:
t = tarfile.open('aclImdb_v1.tar.gz', 'r').extractall('.') 

aclImdb/  aclImdb_v1.tar.gz  imdb


In [42]:
def _read_text_file(path):

    with open(path, 'rt',encoding="utf8") as file:
        
        lines = file.readlines()

        text = " ".join(lines)

    return text


In [52]:
pathlist_pos = Path('/home/dsxuser/work/aclImdb/train/pos').rglob('*.txt')
data_pos = [_read_text_file(path) for path in pathlist_pos]
pathlist_neg = Path('/home/dsxuser/work/aclImdb/train/neg').rglob('*.txt')
data_neg=[_read_text_file(path) for path in pathlist_neg]

In [53]:
x_train_text = data_pos + data_neg
y_train = [1.0] * len(data_pos) + [0.0] * len(data_neg)

In [54]:
pathlist_pos_test = Path('/home/dsxuser/work/aclImdb/test/pos').rglob('*.txt')
data_pos_test = [_read_text_file(path) for path in pathlist_pos_test]
pathlist_neg_test = Path('/home/dsxuser/work/aclImdb/test/neg').rglob('*.txt')
data_neg_test=[_read_text_file(path) for path in pathlist_neg_test]

In [55]:
x_test_text = data_pos_test + data_neg_test
y_test = [1.0] * len(data_pos_test) + [0.0] * len(data_neg_test)

In [73]:
x_train_text[1]

'I voted 8 for this movie because of some minor childish flaws. Other than that, this movie is one of my favorites! It\'s entertaining to say the least. The shooting scenes are ridiculous though, and I think Gackt (who wrote the book) takes a little bit too much of his "Matrix obsession" into it. It seems like their enemies just stands there...waiting to get shot at. However, this movie is touching and it always makes me cry. It has a lot of GREAT humor in it so it makes me laugh as well. Gackt is a superb actor I must say..he shows so much emotion. This was Hyde\'s first time acting and he did okay. The role fits him. Wang Lee Hom is absolutely great. The whole cast is what I would say, perfect for this movie. DON\'T MISS IT! YOU\'LL REGRET IT!'

# Combining the data sets and removing stop words 


In [76]:
# As we can see the data has lot of stop words removing stop words before processing using keras
def filter_stop_words(train_sentences, stop_words):
    for i, sentence in enumerate(train_sentences):
        new_sent = [word for word in sentence.split() if word not in stop_words]
        train_sentences[i] = ' '.join(new_sent)
    return train_sentences



NameError: name 'train_sentences' is not defined

In [77]:
stop_words = set(stopwords.words("english"))
x_train_text = filter_stop_words(x_train_text, stop_words)
x_test_text = filter_stop_words(x_test_text, stop_words)
data_all = x_train_text + x_test_text

In [78]:
num_words = 1000

# Tokenise the train data set using Keras

In [79]:
from keras.models import Sequential
from keras.layers import Dense, GRU, Embedding,LSTM
from keras.optimizers import Adam
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [80]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from scipy.spatial.distance import cdist

# Create a tokeniser using keras

Preparing the Embedding Layer
As a first step, we will use the Tokenizer class from the keras.preprocessing.text module to create a word-to-index dictionary. In the word-to-index dictionary, each word in the corpus is used as a key, while a corresponding unique index is used as the value for the key.

In [81]:
#Create Vocab to Int mapping dictionary using Toeknizer from keras

tokenizer = Tokenizer(num_words=num_words)

In [82]:
## Fit the tokenizer on all the  data

tokenizer.fit_on_texts(data_all)

In [83]:
# Get the word index from the train data 
tokenizer.word_index

{'br': 1,
 'i': 2,
 'the': 3,
 'movie': 4,
 'film': 5,
 'one': 6,
 'it': 7,
 'like': 8,
 'this': 9,
 'good': 10,
 'time': 11,
 'even': 12,
 'would': 13,
 'really': 14,
 'see': 15,
 'story': 16,
 'and': 17,
 'well': 18,
 'much': 19,
 'bad': 20,
 'get': 21,
 'great': 22,
 'also': 23,
 'people': 24,
 'first': 25,
 'but': 26,
 'in': 27,
 'made': 28,
 'make': 29,
 'a': 30,
 'way': 31,
 'movies': 32,
 'could': 33,
 'think': 34,
 'characters': 35,
 'watch': 36,
 'films': 37,
 'two': 38,
 'many': 39,
 'seen': 40,
 'character': 41,
 'there': 42,
 'never': 43,
 'plot': 44,
 'love': 45,
 'acting': 46,
 'life': 47,
 'best': 48,
 "it's": 49,
 'know': 50,
 'show': 51,
 'little': 52,
 'ever': 53,
 'if': 54,
 'all': 55,
 'better': 56,
 'end': 57,
 'man': 58,
 'scene': 59,
 'still': 60,
 'say': 61,
 'scenes': 62,
 'he': 63,
 'that': 64,
 'something': 65,
 'go': 66,
 'back': 67,
 'you': 68,
 'real': 69,
 "i'm": 70,
 'watching': 71,
 'thing': 72,
 'actors': 73,
 'though': 74,
 'funny': 75,
 'years': 76,


# Getting the tokens from Train Data

In [84]:
x_train_tokens = tokenizer.texts_to_sequences(x_train_text)

In [85]:
np.array(x_train_tokens[1])

array([  2, 670,   4, 467,  64,   4,   6,  49, 366,  61, 147,   3,  62,
       589,  74,   2,  34, 308, 205, 237,  52, 149,  19,   7,   7, 106,
         8,  42,  21, 250, 228, 110,   4, 133,  86,   7,  92,  22, 402,
        86, 360,  18, 863, 223,   2, 128,  61,  63, 206,  19,   9,  25,
        11,  46, 746,   3, 135, 185, 909, 356,  22,   3, 146,  96,   2,
        13,  61, 336,   4, 374, 645,   7,   7])

In [86]:
x_test_tokens = tokenizer.texts_to_sequences(x_test_text)

We set the maximum size of each list to 250. The lists with size greater than 250 will be truncated to 250.This process is called padding.

In [89]:
max_len = 250
X_train = pad_sequences(x_train_tokens, maxlen=max_len)

In [90]:
X_test = pad_sequences(x_test_tokens, maxlen=max_len)

In [93]:
X_train.shape

(25000, 250)

# Creating an LTSM model using Keras

In [103]:
model = Sequential()
embedding_size = 8
model.add(Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    input_length=max_len,
                    name='layer_embedding'))

In [104]:
model.add(LSTM(units=16, return_sequences=True))
model.add(LSTM(units=8, return_sequences=True))
model.add(LSTM(units=4))
model.add(Dense(1))

In [105]:
optimizer = Adam(lr=1e-3)
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])


To compile our model, we will use the adam optimizer, binary_crossentropy as our loss function and accuracy as metrics and then we will print the summary of our model:

In [106]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer_embedding (Embedding)  (None, 250, 8)            8000      
_________________________________________________________________
lstm_6 (LSTM)                (None, 250, 16)           1600      
_________________________________________________________________
lstm_7 (LSTM)                (None, 250, 8)            800       
_________________________________________________________________
lstm_8 (LSTM)                (None, 4)                 208       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 5         
Total params: 10,613
Trainable params: 10,613
Non-trainable params: 0
_________________________________________________________________


The model is configured to train on 23750 samples and validate on 1250 samples

In [112]:
history = model.fit(X_train, y_train,
          validation_split=0.05, epochs=1, batch_size=64)

Train on 23750 samples, validate on 1250 samples
Epoch 1/1
23750/23750 [==============================] - 1124s 47ms/step - loss: 0.3889 - acc: 0.8329 - val_loss: 0.5020 - val_acc: 0.7376


To evaluate the performance of the model, we can simply pass the test set to the evaluate method of our model.

In [113]:
result = model.evaluate(X_test, y_test)
print("Accuracy: {0:.2%}".format(result[1]))

25000/25000 [==============================] - 44s 2ms/step
Accuracy: 82.65%


To check the test accuracy and loss, execute the following script:

In [114]:
print("Test Score:", result[0])
print("Test Accuracy:", result[1])

Test Score: 0.37567781929016114
Test Accuracy: 0.82652


We get a test accuracy of 82.652%. Our training accuracy was 83.29%. This means that our model is correctly on the training set. 
The performance difference between training and test sets should be minimum.


In [119]:
history.history

{'val_loss': [0.50202592086792],
 'val_acc': [0.7376000002861023],
 'loss': [0.38887465180848774],
 'acc': [0.8328842105313351]}

Making Predictions on Single Instance

Let us test the predictions aganist 5000 rows from test data 

Prediction for Test Data

In [138]:
y_pred = model.predict(x=X_test[0:5000])
y_pred = y_pred.T[0]

These predicted numbers would be between 0.0 and 1.0. So we set a threshold value of 0.5,and say that all values above 0.5 are taken to be 1.0 and all values below 0.5 are taken to be 0.0. This gives us predicted "class" of either 0.0 or 1.0.

In [139]:
cls_pred = np.array([1.0 if p>0.5 else 0.0 for p in y_pred])

In [140]:
cls_true = np.array(y_test[0:5000])

In [141]:
incorrect = np.where(cls_pred != cls_true)
incorrect = incorrect[0]

In [142]:
len(incorrect)

334

The Model predicted 334 rows incorrectly

In [147]:
idx = incorrect[0]
idx

17

Sample mis-classified text is:

In [148]:
text = x_test_text[idx]
text

"seriously people need lighten accept funny funny, movie f**king hilarious. Better first Knoxville really grew pair film way crazier stunts first. If Ebert Roper(not saying I'm huge fan theirs) look past pure idiocy film enough give 2 thumbs think people to. I sure expect floored rare sequel better original. This new one I believe exceeds first big time. did,just relax kick back try barf points laugh ass off."

The predicted value of sentiment is

In [151]:
y_pred[idx]

0.28527364

In [150]:
cls_true[idx]

1.0